In [2]:
import pandas as pd
import numpy as np
import nbimporter
from datetime import datetime, timedelta
import glob
import os
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# 1. Importieren der Daten

In [1]:
def importing_climate_data(temp_path, temp_file, column_name, date_format):
    """
    Importiert Klimadaten aus CSV-Dateien und speichert sie in einem Dictionary, wobei die Daten nach Jahren gefiltert werden.

    Parameter:
    temp_path (str): Der Pfad zum Verzeichnis, das die CSV-Dateien enthält.
    temp_file (str): Das Dateimuster zur Auswahl der CSV-Dateien.
    column_name (str): Der Name der Spalte in den CSV-Dateien, die das Datum enthält.
    date_format (str): Das Datumsformat, das zum Parsen der Datumsspalte verwendet wird.

    Rückgabewert:
    dict: Ein Wörterbuch, in dem die Schlüssel die Jahre der Interessensjahre sind und die Werte die DataFrames mit den gefilterten Daten für jedes Jahr.
    """
    
    # Liste der Jahre, die von Interesse sind
    years_of_interest = [2016, 2018, 2021, 2024]
    
    # Suchen der CSV-Dateien im angegebenen Verzeichnis
    temp_file = glob.glob(os.path.join(temp_path, temp_file))
    
    df_dic = {}
    
    for file in temp_file:
        # Laden der CSV-Datei in einen DataFrame
        df = pd.read_csv(file, delimiter=";")
        
        # Umwandeln der Datumsangaben in datetime-Objekte
        df["DateTime"] = pd.to_datetime(df[column_name], format=date_format)
        
        for year in years_of_interest:
            # Filtern der Daten für das aktuelle Jahr
            data = df[df["DateTime"].dt.year == year]
            
            if not data.empty:
                # Umbenennen der Spalten, um die Station zu kennzeichnen
                exclude_col = "DateTime"
                data.columns = [station + "_" + col if col != exclude_col else col for col in data.columns]
                df_dic[year] = data
    
    return df_dic


In [2]:
def importing_football_data(temp_path, temp_file, column_name, *date_format):
    """
    Importiert Fußballspieldaten aus CSV-Dateien und speichert sie in einem Dictionary, wobei die Daten nach Jahren gefiltert werden.

    Parameter:
    temp_path (str): Der Pfad zum Verzeichnis, das die CSV-Dateien enthält.
    temp_file (str): Das Dateimuster zur Auswahl der CSV-Dateien.
    column_name (str): Der Name der Spalte in den CSV-Dateien, die das Datum enthält.
    *date_format (str): Ein oder zwei Datumsformate, die zum Parsen der Datumsspalte verwendet werden. 
                        Falls das erste Format fehlschlägt, wird das zweite Format verwendet.

    Rückgabewert:
    dict: Ein Wörterbuch, in dem die Schlüssel die Jahre der Interessensjahre sind und die Werte die DataFrames mit den gefilterten Daten für jedes Jahr.
    """
    
    # Suchen der CSV-Dateien im angegebenen Verzeichnis
    temp_file = glob.glob(os.path.join(temp_path, temp_file))
    
    df_dic = {}
    years_of_interest = [2016, 2018, 2021, 2024]
    
    for file in temp_file:
        # Laden der CSV-Datei in einen DataFrame
        df = pd.read_csv(file, delimiter=";")
        
        # Erstellen einer DateTime-Spalte aus Datum und Uhrzeit
        df["DateTime"] = df["Datum"] + " " + df["Uhrzeit (MESZ)"]

        # Festlegen der Liga basierend auf dem Dateinamen
        if "EM" in file:
            df["Liga"] = "EM"
        elif "WM" in file:
            df["Liga"] = "WM"
        
        try:
            # Versuchen, das Datum mit dem ersten Format zu parsen
            df["DateTime"] = pd.to_datetime(df[column_name], format=date_format[0])
        except:
            # Falls das erste Format fehlschlägt, das zweite Format verwenden
            df["DateTime"] = pd.to_datetime(df[column_name], format=date_format[1])

        for year in years_of_interest:
            # Filtern der Daten für das aktuelle Jahr
            data = df[df["DateTime"].dt.year == year]

            if not data.empty:
                df_dic[year] = data
    
    return df_dic


In [3]:
def importing_electricity_data(temp_path, file_name, date_column, date_format):
    """
    Importiert Stromverbrauchsdaten aus CSV-Dateien und organisiert sie nach Jahren in einem Dictionary.

    Parameter:
    temp_path (str): Der Pfad zum Verzeichnis, das die CSV-Dateien enthält.
    file_name (str): Das Dateimuster zur Auswahl der CSV-Dateien.
    date_column (str): Der Name der Spalte in den CSV-Dateien, die das Datum enthält.
    date_format (str): Das Datumsformat, das zum Parsen der Datumsspalte verwendet wird.

    Rückgabewert:
    dict: Ein Wörterbuch, in dem die Schlüssel die Jahre sind und die Werte die DataFrames mit den gefilterten Stromverbrauchsdaten für jedes Jahr.
    """
    
    csv_files = glob.glob(os.path.join(temp_path, file_name))
    
    years_of_interest = [2016, 2018, 2021, 2024]
    stromverbrauch_list = []
    
    for file in csv_files:
        # Laden der CSV-Datei in einen DataFrame
        df = pd.read_csv(file, delimiter=";")
        
        # Umwandeln der Datumsspalte in datetime-Objekte
        df["DateTime"] = pd.to_datetime(df[date_column], format=date_format)
    
        for year in years_of_interest:
            # Filtern der Daten für das aktuelle Jahr
            data = df[df["DateTime"].dt.year == year]
            data.set_index("DateTime", inplace=True)
            
            if not data.empty:
                # Bereinigen der Stromverbrauchsspalte und Konvertieren in float
                df['Gesamt (Netzlast) [MWh] Originalauflösungen'] = df['Gesamt (Netzlast) [MWh] Originalauflösungen'].str.replace('.', '', regex=False)
                df['Gesamt (Netzlast) [MWh] Originalauflösungen'] = df['Gesamt (Netzlast) [MWh] Originalauflösungen'].str.replace(',', '.', regex=False)
                df['Gesamt (Netzlast) [MWh] Originalauflösungen'] = df['Gesamt (Netzlast) [MWh] Originalauflösungen'].astype(float)
                stromverbrauch_list.append(df)
    
    # Erstellen einer Liste von Tupeln (Datum, DataFrame) und Sortieren nach dem frühesten Datum
    df_tuples = [(df["DateTime"].iloc[0], df) for df in stromverbrauch_list]
    df_tuples_sorted = sorted(df_tuples, key=lambda x: x[0])
    
    elec_dic = {}
    
    # Zusammenführen der DataFrames nach Jahr
    for dt, df in df_tuples_sorted:
        year = dt.year
        
        if year not in elec_dic:
            elec_dic[year] = df
        else:
            elec_dic[year] = pd.concat([elec_dic[year], df], ignore_index=True)
    
    return elec_dic


# 2. Manipulation der Dataframes

In [4]:
def resampling_data(required_dic, resample_step):
    """
    Resampelt Zeitreihendaten in einem Dictionary nach einem angegebenen Zeitintervall.

    Parameter:
    required_dic (dict): Ein Wörterbuch, in dem die Schlüssel die Jahre oder andere Bezeichner sind und die Werte DataFrames mit den Zeitreihendaten.
    resample_step (str): Das Zeitintervall für das Resampling

    Rückgabewert:
    dict: Ein Wörterbuch, in dem die Schlüssel die gleichen wie in `required_dic` sind und die Werte DataFrames mit den resampleten Zeitreihendaten.
    """
    
    resampled_dic = {}
    
    for key, climate_df in required_dic.items():
        try:
            # Setzen des Index auf die Datetime-Spalte
            climate_df.set_index("DateTime", inplace=True)
        except:
            pass
        
        # Resampling der numerischen Spalten mit Mittelwert
        numeric_cols = climate_df.select_dtypes(include='number').columns
        df_resampled_numeric = climate_df[numeric_cols].resample(resample_step).mean()
    
        # Resampling der nicht-numerischen Spalten mit dem häufigsten Wert (Mode)
        non_numeric_cols = climate_df.select_dtypes(exclude='number').columns
        df_resampled_non_numeric = climate_df[non_numeric_cols].resample(resample_step).agg(lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0])
    
        # Kombinieren der resampleten numerischen und nicht-numerischen DataFrames
        df_resampled = pd.concat([df_resampled_numeric, df_resampled_non_numeric], axis=1)
    
        resampled_dic[key] = df_resampled

    return resampled_dic


In [5]:
def new_col_match(match_set):
    """
    Verarbeitet ein DataFrame von Fußballspielen und erstellt neue Zeilen, wenn mehrere Spiele für das gleiche Datum vorhanden sind.

    Parameter:
    match_set (pd.DataFrame): Ein DataFrame, das die Spalten "Land1", "Land2", "Runde", "Liga" und "DateTime" enthält. 

    Rückgabewert:
    pd.DataFrame: Ein DataFrame, das die verarbeiteten Daten enthält, bei denen für jedes Datum entweder die Originalzeile oder eine neue Zeile für mehrere Spiele erstellt wurde.
    """
    
    # Entfernen von Leerzeichen in den Ländernamen
    match_set["Land1"] = match_set["Land1"].str.strip()
    match_set["Land2"] = match_set["Land2"].str.strip()
    
    # Auswahl der relevanten Spalten
    match_set = match_set.loc[:, ["Land1", "Land2", "Runde", "Liga", "DateTime"]]
    
    # Gruppieren nach Datum
    grouped = match_set.groupby("DateTime")
    results = pd.DataFrame()

    for name, group in grouped:
        if len(group) == 1:
            # Falls nur ein Spiel für das Datum vorhanden ist, hinzufügen
            results = pd.concat([results, group])
        else:
            # Falls mehrere Spiele vorhanden sind, eine neue Zeile erstellen
            land1_value = group.iloc[0]['Land1']
            land2_value = group.iloc[0]['Land2']

            new_row = {
                "DateTime": name,
                'Land1': land1_value,
                'Land2': land2_value,
                "Runde": group.iloc[1]["Runde"],
                "Liga": group.iloc[1]["Liga"],
                'Land3': group.iloc[1]['Land1'],
                'Land4': group.iloc[1]['Land2'],
            }

            new_df = pd.DataFrame([new_row])
            results = pd.concat([results, new_df], ignore_index=True)
    
    return results


# 3. Merging Dataframes

In [6]:
def merging_data(electricity_data, football_data, climate_data, unwantend_col_string):
    """
    Fügt verschiedene Datensätze zu einem einzigen DataFrame zusammen und füllt fehlende Werte basierend auf den Spielzeiten auf.

    Parameter:
    electricity_data (pd.DataFrame): DataFrame mit Stromverbrauchsdaten, der eine "DateTime"-Spalte enthält.
    football_data (pd.DataFrame): DataFrame mit Fußballspieldaten, der eine "DateTime"-Spalte enthält.
    climate_data (list of pd.DataFrame): Eine Liste von DataFrames mit Klimadaten, die jeweils eine "DateTime"-Spalte enthalten.
    unwantend_col_string (list of str): Eine Liste von Strings, die in den Spaltennamen der nicht gewünschten Spalten enthalten sind.

    Rückgabewert:
    pd.DataFrame: Ein DataFrame, das die zusammengeführten Daten enthält, bei dem fehlende Werte basierend auf den Spielzeiten aufgefüllt und unerwünschte Spalten entfernt wurden.
    """
    
    # Erstellen einer Liste von DataFrames, die zusammengeführt werden sollen
    dfs_year = [electricity_data, football_data] + climate_data
    
    start_elec = electricity_data["DateTime"].iloc[0]
    end_elec = electricity_data["DateTime"].iloc[-1]
    
    # Zusammenführen der DataFrames
    merged_df = reduce(lambda left, right: pd.merge(left, right, on='DateTime', how='outer'), dfs_year)
    
    # Auflisten der Spaltennamen des Fußball-DataFrames
    spielplan_name_list = football_data.columns.tolist()

    # Fehlende Werte in den Zeiträumen der Fußballspiele auffüllen
    for index, row in football_data.iterrows():
        start_time = row["DateTime"]
        until_time = row["DateTime"] + timedelta(minutes=105)
        fill_condition = (merged_df['DateTime'] >= start_time) & (merged_df['DateTime'] <= until_time)
        merged_df.loc[fill_condition, spielplan_name_list] = merged_df.loc[fill_condition, spielplan_name_list].fillna(method='ffill')

    # Entfernen von unerwünschten Spalten
    for string in unwantend_col_string:
        merged_df = merged_df.drop(columns=merged_df.filter(like=string).columns)
        
    # Einschränken des DataFrames auf den Zeitraum der Stromverbrauchsdaten
    merged_df = merged_df[(merged_df["DateTime"] >= start_elec) & (merged_df["DateTime"] <= end_elec)]

    return merged_df
